In [2]:
import ee
import geemap 
import geopandas as gpd
from datetime import datetime
import numpy as np
import rasterio
import pandas as pd
import requests


In [3]:
ee.Authenticate(force=True)
ee.Initialize()


Successfully saved authorization token.


### Download Sentinel-1 Images

In [ ]:

def load_shapefile(file_path):
    gdf = gpd.read_file(file_path)
    geom = gdf.geometry.union_all()
    return ee.Geometry(geom.__geo_interface__)  # convert to ee.Geometry

# Function to check if an image has >99% valid pixels
def is_valid_image(image, roi):
    mask = image.mask().clip(roi)
    valid_pixel_count = mask.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=10,
        maxPixels=1e13
    ).values().get(0)
    
    total_pixel_count = ee.Number(image.unmask().clip(roi).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=roi,
        scale=10,
        maxPixels=1e13
    ).values().get(0))
    
    valid_ratio = ee.Number(valid_pixel_count).divide(total_pixel_count).multiply(100)
    return valid_ratio.gte(99)  # True if ≥99% valid pixels

# Function to get the nearest Sentinel-1 image with specified orbit pass
def get_nearest_sentinel1(roi, target_date, orbit_pass):
    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(roi) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit_pass))  # Filter ASCENDING or DESCENDING images

    start_date = ee.Date(target_date).advance(-3, 'day')
    end_date = ee.Date(target_date).advance(3, 'day')
    
    filtered = collection.filterDate(start_date, end_date)
    # Check if the filtered collection is empty
    if filtered.size().getInfo() == 0:
        print(f"No images found for date range {start_date.format('YYYY-MM-dd').getInfo()} to {end_date.format('YYYY-MM-dd').getInfo()}")
        return None  # No valid images found

    images = filtered.sort('system:time_start').toList(filtered.size())


    best_image = None
    min_diff = None

    for i in range(images.size().getInfo()):
        img = ee.Image(images.get(i)).select(['VV', 'VH']).toFloat()
        img_date = ee.Date(img.get('system:time_start'))
        diff = img_date.difference(ee.Date(target_date), 'day').abs()
        is_later = img_date.difference(ee.Date(target_date), 'day').gt(0)

        if best_image is None or diff.lt(min_diff).getInfo() or (diff.eq(min_diff).getInfo() and is_later.getInfo()):
            if is_valid_image(img, roi).getInfo():
                best_image = img
                min_diff = diff

    return best_image

# Function to export image to Google Drive
def export_image(image, roi, filename, site):
    task = ee.batch.Export.image.toDrive(
        image=image.clip(roi).toFloat(),
        description=filename,
        folder= site,  # Change folder as needed
        fileNamePrefix=filename,
        scale=10,
        region=roi.bounds(),
        crs = 'EPSG:4326',   
        maxPixels = 1e13,
        fileFormat='GeoTIFF'
    )
    task.start()
    print(f"Export started: {filename}")

# Example usage
# SCAN_Kemole_Gulch
# SCAN_Kukuihaele
# Mana_House
# SCAN_Silver_Sword
# SCAN_Waimea_Plain
# TAHMO_CRIG_(Soil_Moisture_Station_1)
# TAHMO_CRIG_(Soil_Moisture_Station_2)
site = 'TAHMO_CRIG_(Soil_Moisture_Station_2)'
shapefile_path = f"shapefiles/{site}_smap/{site}_smap_polygons.shp"
csv_path = f"daily_smap/{site}_smap.csv"
roi = load_shapefile(shapefile_path)
dates = []  # List of target dates

df = pd.read_csv(csv_path)
valid_dates = df.dropna(subset=['smap_soil_moisture_am'])

# valid_dates
for index, row in valid_dates.iterrows():
    dates.append(row['time'])
print(len(dates))
for date in dates:
    for orbit in ['DESCENDING', 'ASCENDING']:
        image = get_nearest_sentinel1(roi, date, orbit)
        if image:
            print(f"Found Sentinel-1 {orbit} image for {date}")
            export_image(image, roi, f"{site}_S1_{date}_{orbit}", site)
        else:
            print(f"No valid {orbit} Sentinel-1 data from {date}-3 to {date}+3 days.")


### Download NDVI 10 m calculated from Sentinel-2

In [18]:
import ee
import geopandas as gpd
import pandas as pd
import os

# Load shapefile as ROI
def load_shapefile(file_path):
    gdf = gpd.read_file(file_path)
    geom = gdf.geometry.unary_union
    return ee.Geometry(geom.__geo_interface__)

def is_valid(image, roi, threshold = 0.5):
    valid_pixel_mask = image.mask().reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = roi,
        scale = 10,
        bestEffort = True
    ).getNumber('NDVI')

    # Count total pixels 
    total_pixels = image.unmask().reduceRegion(
        reducer = ee.Reducer.count(),
        geometry = roi,
        scale = 10,
        bestEffort = True
    ).getNumber('NDVI')

    fraction_valid = valid_pixel_mask.divide(total_pixels)

    return fraction_valid.gte(threshold)



# Get Sentinel-2 collection for a specific target_day
def get_sentinel_collection(target_day, roi):
    s_date = ee.Date(target_day).advance(-4, 'day')
    e_date = ee.Date(target_day).advance(4, 'day')

    cs_plus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
    qa_band = 'cs'
    clear_threshold = 0.5

    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
                    .filterBounds(roi)\
                    .filterDate(s_date, e_date)\
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))\
                    .linkCollection(cs_plus, [qa_band])

    cloud_masked_sentinel2 = sentinel2.map(lambda img: img.updateMask(img.select(qa_band).gte(clear_threshold)).clip(roi))

    return cloud_masked_sentinel2  

# Separate Sentinel collection into Cloud-Free and Cloudy collections
def separate_collections(ndvi_collection):
    cloud_free_collection = ndvi_collection.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))
    cloudy_collection = ndvi_collection.filter(ee.Filter.gt('CLOUDY_PIXEL_PERCENTAGE', 10))
    return {'cloudFree': cloud_free_collection, 'cloudy': cloudy_collection}

# def compute_mean_ndvi(image_collection):
#     ndvi_collection = image_collection.map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'))
#     return ndvi_collection.mean()


def compute_mean_ndvi(image_collection):
    """Compute mean NDVI and handle empty collections."""
    has_images = image_collection.size().gt(0)

    def compute():
        mean_nvdi = image_collection.mean().normalizedDifference(['B8', 'B4']).rename('NDVI')
    
        # ndvi_collection = image_collection.map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'))
        # valid_pixel_count = ndvi_collection.count()
        # mean_nvdi = mean_nvdi.updateMask(valid_pixel_count.gt(0))
        return mean_nvdi

    ndvi_image = ee.Algorithms.If(has_images, compute(), None)
    return ee.Image(ndvi_image) if ndvi_image else None 

# Download image to local storage
def download_image(image, ee_geom, output_filepath):
    """
    Download a GEE image using getDownloadURL.
    :param image: ee.Image to download.
    :param ee_geom: ee.Geometry used as region.
    :param output_filepath: Local filepath to save the image.
    """
    if image is None:
        print("Error: The image is None. Skipping download.")
        return
    
    try:
        # Ensure image is not empty before downloading
        image_size = image.bandNames().size().getInfo()
        if image_size == 0:
            print("Error: The image has no bands. Skipping download.")
            return
        
        # Replace no-data pixels with nodata_value
        nodata_value = -9999
        image = image.unmask(nodata_value)

        params = {
            'scale': 10,
            'region': ee_geom.bounds().getInfo(),
            'filePerBand': False,
            'format': 'GeoTIFF'
        }
        
        download_url = image.getDownloadURL(params)
        print(f"Downloading from URL: {download_url}")
        response = requests.get(download_url, stream=True)

        if response.status_code == 200:
            temp_filepath = output_filepath + '.tmp'
            with open(temp_filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
            print(f"Downloaded image saved to {temp_filepath}")
        
            with rasterio.open(temp_filepath) as src:
                data = src.read(1)
                data[data == nodata_value] = np.nan 
                profile = src.profile

            profile.update(nodata = np.nan)

            with rasterio.open(output_filepath, 'w', **profile) as dst:
                dst.write(data, 1)

            os.remove(temp_filepath)
            print(f"Converted no-data values to NaN and saved to {output_filepath}")
        else:
            print(f"Failed to download image, status code: {response.status_code}")

    except Exception as e:
        print(f"Error downloading image: {e}")

# Main execution
site = 'TAHMO_CRIG_(Soil_Moisture_Station_2)'
shapefile_path = f"shapefiles/{site}_smap/{site}_smap_polygons.shp"
roi = load_shapefile(shapefile_path)
csv_path = f"daily_smap/{site}_smap.csv"

df = pd.read_csv(csv_path)
valid_dates = df.dropna(subset=['smap_soil_moisture_am'])
dates = valid_dates['time'].tolist()
os.makedirs(f"./downloads1/{site}", exist_ok=True)
for target_day in dates:
    print(f"Processing {target_day}...")

    # Load Sentinel-2 and separate collections
    sentinel_collection = get_sentinel_collection(target_day, roi)
    collections = separate_collections(sentinel_collection)

    # Compute NDVI for each collection
    ndvi_cloudFree = compute_mean_ndvi(collections['cloudFree'])
    if ndvi_cloudFree:
        download_image(ndvi_cloudFree, roi, f"downloads1/{site}/ndvi_noncloud_{site}_{target_day}.tif")
    ndvi_cloudy = compute_mean_ndvi(collections['cloudy'])
    if ndvi_cloudy:
        if is_valid(ndvi_cloudy, roi):
            download_image(ndvi_cloudy, roi, f"downloads1/{site}/ndvi_cloudy_{site}_{target_day}.tif")
        else:
            print("Skipping cloudy image {date} due to low valid pixel count.")

print("All downloads completed.")


/tmp/ipykernel_5166/1535105245.py:9: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = gdf.geometry.unary_union


Processing 2020-01-02...
Downloaded image saved to downloads1/TAHMO_CRIG_(Soil_Moisture_Station_2)/ndvi_noncloud_TAHMO_CRIG_(Soil_Moisture_Station_2)_2020-01-02.tif.tmp
Converted no-data values to NaN and saved to downloads1/TAHMO_CRIG_(Soil_Moisture_Station_2)/ndvi_noncloud_TAHMO_CRIG_(Soil_Moisture_Station_2)_2020-01-02.tif
Error downloading image: Image.bandNames: Parameter 'image' is required and may not be null.
Processing 2020-01-10...
Downloaded image saved to downloads1/TAHMO_CRIG_(Soil_Moisture_Station_2)/ndvi_noncloud_TAHMO_CRIG_(Soil_Moisture_Station_2)_2020-01-10.tif.tmp
Converted no-data values to NaN and saved to downloads1/TAHMO_CRIG_(Soil_Moisture_Station_2)/ndvi_noncloud_TAHMO_CRIG_(Soil_Moisture_Station_2)_2020-01-10.tif
Error downloading image: Image.bandNames: Parameter 'image' is required and may not be null.
Processing 2020-01-18...
Downloaded image saved to downloads1/TAHMO_CRIG_(Soil_Moisture_Station_2)/ndvi_noncloud_TAHMO_CRIG_(Soil_Moisture_Station_2)_2020-01-

### Download NVDI

In [ ]:

def load_shapefile(file_path):
    gdf = gpd.read_file(file_path)
    geom = gdf.geometry.unary_union
    return ee.Geometry(geom.__geo_interface__)  # convert to ee.Geometry

# Function to check if an image has >99% valid pixels
def is_valid_image(image, roi):
    mask = image.mask().clip(roi)
    valid_pixel_count = mask.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=500,
        maxPixels=1e13
    ).values().get(0)
    
    total_pixel_count = ee.Number(image.unmask().clip(roi).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=roi,
        scale=500,
        maxPixels=1e13
    ).values().get(0))
    
    valid_ratio = ee.Number(valid_pixel_count).divide(total_pixel_count).multiply(100)
    return valid_ratio.gte(99)  # True if ≥99% valid pixels

# Function to get the nearest Sentinel-1 image with specified orbit pass
def get_nearest_viirs(roi, target_date):
    collection = ee.ImageCollection('NASA/VIIRS/002/VNP13A1') \
        .filterBounds(roi)

    start_date = ee.Date(target_date).advance(-4, 'day')
    end_date = ee.Date(target_date).advance(4, 'day')
    
    filtered = collection.filterDate(start_date, end_date)
    # Check if the filtered collection is empty
    if filtered.size().getInfo() == 0:
        print(f"No images found for date range {start_date.format('YYYY-MM-dd').getInfo()} to {end_date.format('YYYY-MM-dd').getInfo()}")
        return None  # No valid images found

    images = filtered.sort('system:time_start').toList(filtered.size())


    best_image = None
    min_diff = None

    for i in range(images.size().getInfo()):
        img = ee.Image(images.get(i)).select('NDVI').toFloat()
        img_date = ee.Date(img.get('system:time_start'))
        diff = img_date.difference(ee.Date(target_date), 'day').abs()
        is_later = img_date.difference(ee.Date(target_date), 'day').gt(0)

        if best_image is None or diff.lt(min_diff).getInfo() or (diff.eq(min_diff).getInfo() and is_later.getInfo()):
            if is_valid_image(img, roi).getInfo():
                best_image = img
                min_diff = diff

    return best_image

# Function to export image to Google Drive
def export_image(image, roi, filename, site):
    task = ee.batch.Export.image.toDrive(
        image=image.toFloat(),
        description=filename,
        folder= 'NDVI_'+site,  # Change folder as needed
        fileNamePrefix=filename,
        scale=500,
        region=roi.bounds(),
        crs = 'EPSG:4326',   
        maxPixels = 1e13,
        fileFormat='GeoTIFF'
    )
    task.start()
    print(f"Export started: {filename}")

# Example usage
# SCAN_Kemole_Gulch oke
# SCAN_Kukuihaele none
# SCAN_Mana_House oke
# SCAN_Silver_Sword
# SCAN_Waimea_Plain
# TAHMO_CRIG_(Soil_Moisture_Station_1)
# TAHMO_CRIG_(Soil_Moisture_Station_2)
site = 'SCAN_Silver_Sword'
shapefile_path = f"shapefiles/{site}_smap/{site}_smap_polygons.shp"
csv_path = f"daily_smap/{site}_smap.csv"
roi = load_shapefile(shapefile_path)
dates = []  # List of target dates

df = pd.read_csv(csv_path)
valid_dates = df.dropna(subset=['smap_soil_moisture_am'])

for index, row in valid_dates.iterrows():
    dates.append(row['time'])

print(len(dates))

for date in dates:
    image = get_nearest_viirs(roi, date)
    if image:
        print(f"Found VIIRS image for {date}")
        export_image(image, roi, f"{site}_VIIRS_{date}", site)
    else:
            print(f"No valid VIIRS data from {date}-4 to {date}+4 days.")


/tmp/ipykernel_14644/1394914834.py:3: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = gdf.geometry.unary_union


176
Found VIIRS image for 2020-01-06
Export started: SCAN_Silver_Sword_VIIRS_2020-01-06
Found VIIRS image for 2020-01-09
Export started: SCAN_Silver_Sword_VIIRS_2020-01-09
Found VIIRS image for 2020-01-14
Export started: SCAN_Silver_Sword_VIIRS_2020-01-14
Found VIIRS image for 2020-01-17
Export started: SCAN_Silver_Sword_VIIRS_2020-01-17
Found VIIRS image for 2020-01-22
Export started: SCAN_Silver_Sword_VIIRS_2020-01-22
Found VIIRS image for 2020-01-25
Export started: SCAN_Silver_Sword_VIIRS_2020-01-25
Found VIIRS image for 2020-01-30
Export started: SCAN_Silver_Sword_VIIRS_2020-01-30
Found VIIRS image for 2020-02-02
Export started: SCAN_Silver_Sword_VIIRS_2020-02-02
Found VIIRS image for 2020-02-07
Export started: SCAN_Silver_Sword_VIIRS_2020-02-07
Found VIIRS image for 2020-02-10
Export started: SCAN_Silver_Sword_VIIRS_2020-02-10
Found VIIRS image for 2020-02-15
Export started: SCAN_Silver_Sword_VIIRS_2020-02-15
Found VIIRS image for 2020-02-18
Export started: SCAN_Silver_Sword_VIIRS

In [5]:

def load_shapefile(file_path):
    gdf = gpd.read_file(file_path)
    geom = gdf.geometry.unary_union
    return ee.Geometry(geom.__geo_interface__)  # convert to ee.Geometry

# Function to export image to Google Drive
def export_image(image, roi, filename, site):
    task = ee.batch.Export.image.toDrive(
        image=image.toFloat(),
        description=filename,
        folder= 'NDVI_'+site+"2021",  # Change folder as needed
        fileNamePrefix=filename,
        scale=500,
        region=roi.bounds(),
        crs = 'EPSG:4326',   
        maxPixels = 1e13,
        fileFormat='GeoTIFF'
    )
    task.start()
    print(f"Export started: {filename}")

# Example usage
# SCAN_Kemole_Gulch oke
# SCAN_Kukuihaele none
# SCAN_Mana_House oke
# SCAN_Silver_Sword
# SCAN_Waimea_Plain
# TAHMO_CRIG_(Soil_Moisture_Station_1)
# TAHMO_CRIG_(Soil_Moisture_Station_2)
site = 'SCAN_Silver_Sword'
shapefile_path = f"shapefiles/{site}_smap/{site}_smap_polygons.shp"
csv_path = f"daily_smap/{site}_smap.csv"
roi = load_shapefile(shapefile_path)

start_date = '2021-01-01'
end_date = '2021-02-28'

nvdi_collection = ee.ImageCollection('NASA/VIIRS/002/VNP13A1') \
                    .filterBounds(roi) \
                    .filterDate(start_date, end_date)

image_list = nvdi_collection.toList(nvdi_collection.size())

for i in range(nvdi_collection.size().getInfo()):
    image = ee.Image(image_list.get(i)).select('NDVI')
    image = image.toFloat()
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    export_image(image, roi, f"{site}_VIIRS_{date}", site)


/tmp/ipykernel_25817/122484514.py:3: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = gdf.geometry.unary_union


Export started: SCAN_Silver_Sword_VIIRS_2021-01-01
Export started: SCAN_Silver_Sword_VIIRS_2021-01-09
Export started: SCAN_Silver_Sword_VIIRS_2021-01-17
Export started: SCAN_Silver_Sword_VIIRS_2021-01-25
Export started: SCAN_Silver_Sword_VIIRS_2021-02-02
Export started: SCAN_Silver_Sword_VIIRS_2021-02-10
Export started: SCAN_Silver_Sword_VIIRS_2021-02-18
Export started: SCAN_Silver_Sword_VIIRS_2021-02-26


### Download LST

In [ ]:
import os
import glob
import requests
import pandas as pd
import geopandas as gpd
import ee
from datetime import datetime, timedelta

# Initialize the Earth Engine API.
ee.Authenticate(force=True)
ee.Initialize(project='ee-hadat')

# Define directories for CSV files and shapefiles.
csv_dir = 'soil_moisture_lst/soil_moisture_lst/daily_smap'
shp_base_dir = 'soil_moisture_lst/soil_moisture_lst/shapefiles'

# New folder for downloaded images.
download_base_dir = 'downloaded_images'
os.makedirs(download_base_dir, exist_ok=True)

# Get list of all CSV files in the csv_dir.
csv_files = glob.glob(os.path.join(csv_dir, '*.csv'))

def shapely_to_ee(shapely_geom):
    """Convert a Shapely geometry to an Earth Engine geometry."""
    if shapely_geom.geom_type == 'Polygon':
        coords = list(shapely_geom.exterior.coords)
        return ee.Geometry.Polygon(coords)
    elif shapely_geom.geom_type == 'MultiPolygon':
        polygons = [ee.Geometry.Polygon(list(poly.exterior.coords)) for poly in shapely_geom.geoms]
        # For simplicity, here we create a MultiPolygon from the coordinates of each polygon.
        return ee.Geometry.MultiPolygon([poly.coordinates().getInfo() for poly in polygons])
    else:
        raise ValueError("Unsupported geometry type: {}".format(shapely_geom.geom_type))

def download_gee_image(image, ee_geom, scale, output_filepath):
    """
    Download a GEE image using getDownloadURL.
    :param image: ee.Image to download.
    :param ee_geom: ee.Geometry used as region.
    :param scale: Scale (in meters) for the download.
    :param output_filepath: Local filepath to save the image.
    """
    params = {
        'scale': scale,
        'region': ee_geom.coordinates().getInfo(),
        'filePerBand': False,
        'format': 'GeoTIFF'
    }
    try:
        download_url = image.getDownloadURL(params)
        print(f"Downloading from URL: {download_url}")
        response = requests.get(download_url, stream=True)
        if response.status_code == 200:
            with open(output_filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
            print(f"Downloaded image saved to {output_filepath}")
        else:
            print(f"Failed to download image, status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading image: {e}")

def download_images_for_region(region_name, ee_geom, unique_dates, scale=1000):
    """
    For each unique date, download one image from each collection for a given region.
    Images are saved in a subfolder named after the region.
    :param region_name: Name of the region.
    :param ee_geom: Earth Engine geometry of the region.
    :param unique_dates: List or array of unique date objects.
    :param scale: Download scale in meters.
    """
    region_folder = os.path.join(download_base_dir, region_name)
    os.makedirs(region_folder, exist_ok=True)

for day in unique_dates:
        start_date = datetime.combine(day, datetime.min.time())
        end_date = start_date + timedelta(days=1)
        start_str = start_date.strftime('%Y-%m-%d')
        end_str = end_date.strftime('%Y-%m-%d')
        
        print(f"Downloading images for region '{region_name}' on {start_str}")
        
        # Filter each image collection by date and geometry.
        collection_d = ee.ImageCollection('NASA/VIIRS/002/VNP21A1D') \
            .filterDate(start_str, end_str) \
            .filterBounds(ee_geom)
        collection_n = ee.ImageCollection('NASA/VIIRS/002/VNP21A1N') \
            .filterDate(start_str, end_str) \
            .filterBounds(ee_geom)
        
        # Get the first image from each collection (if available).
        image_d = collection_d.first()
        image_n = collection_n.first()
        
        if image_d:
            output_file_d = os.path.join(region_folder, f"{start_str}_VNP21A1D.tif")
            download_gee_image(image_d, ee_geom, scale, output_file_d)
        else:
            print(f"  No VNP21A1D image found for {start_str} in region {region_name}")
        
        if image_n:
            output_file_n = os.path.join(region_folder, f"{start_str}_VNP21A1N.tif")
            download_gee_image(image_n, ee_geom, scale, output_file_n)
        else:
            print(f"  No VNP21A1N image found for {start_str} in region {region_name}")

# Main loop: Process each CSV and its corresponding shapefile.
for csv_path in csv_files:
    base_name = os.path.splitext(os.path.basename(csv_path))[0]
    print(f"\nProcessing region: {base_name}")

    # Construct the corresponding shapefile folder path.
    shp_folder = os.path.join(shp_base_dir, base_name)
    shp_files = glob.glob(os.path.join(shp_folder, '*.shp'))
    if not shp_files:
        print(f" - No shapefile found for {base_name}. Skipping.")
        continue
    shp_path = shp_files[0]  # Use the first found shapefile.
    
    # Read CSV and filter rows with valid coordinates.
    df = pd.read_csv(csv_path)
    df = df.dropna(subset=['smap_lon', 'smap_lat'])
    if df.empty:
        print(f" - No valid coordinate rows found in {base_name}.")
        continue

    try:
        df['time'] = pd.to_datetime(df['time'])
    except Exception as e:
        print(f" - Error parsing dates in {base_name}: {e}")
        continue

    unique_dates = df['time'].dt.date.unique()
    print(f" - Found {len(unique_dates)} days with coordinates.")
    
    # Read shapefile using geopandas.
    gdf = gpd.read_file(shp_path)
    union_geom = gdf.unary_union
    try:
        ee_geom = shapely_to_ee(union_geom)
    except Exception as e:
        print(f" - Error converting geometry for {base_name}: {e}")
        continue

    # Optionally: Process images in GEE (e.g., count images per day)
    for day in unique_dates:
        start_date = datetime.combine(day, datetime.min.time())
        end_date = start_date + timedelta(days=1)

In [ ]:
import rasterio
import numpy as np

def split_s1_bands(input_path, vv_output, vh_output):
    """
    Tách bands VV và VH từ ảnh Sentinel-1A
    
    Tham số:
        input_path (str): Đường dẫn đến file ảnh đầu vào
        vv_output (str): Đường dẫn lưu band VV
        vh_output (str): Đường dẫn lưu band VH
    """
    with rasterio.open(input_path) as src:
        # Kiểm tra số lượng bands
        if src.count < 2:
            raise ValueError("Input file phải chứa ít nhất 2 bands")
            
        # Đọc metadata gốc
        meta = src.meta.copy()
        
        # Cập nhật metadata cho từng band
        vv_meta = meta.update({"count": 1, "dtype": "float32"})
        vh_meta = meta.update({"count": 1, "dtype": "float32"})

        # Ghi band VV (band 1)
        with rasterio.open(vv_output, 'w', **vv_meta) as dst:
            dst.write(src.read(1), 1)
            
        # Ghi band VH (band 2) 
        with rasterio.open(vh_output, 'w', **vh_meta) as dst:
            dst.write(src.read(2), 1)

# Cách sử dụng
split_s1_bands(
    input_path='S1A_IW_GRDH_1SDV_20230301T050000.tif',
    vv_output='VV_band.tif',
    vh_output='VH_band.tif'
)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12572\1841385777.py:3: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = gdf.geometry.unary_union


Found Sentinel-1 DESCENDING image for 2020-01-06
Export started: SCAN_Kemole_Gulch_S1_2020-01-06_DESCENDING
Found Sentinel-1 ASCENDING image for 2020-01-06
Export started: SCAN_Kemole_Gulch_S1_2020-01-06_ASCENDING
Found Sentinel-1 DESCENDING image for 2020-01-09
Export started: SCAN_Kemole_Gulch_S1_2020-01-09_DESCENDING
Found Sentinel-1 ASCENDING image for 2020-01-09
Export started: SCAN_Kemole_Gulch_S1_2020-01-09_ASCENDING
Found Sentinel-1 DESCENDING image for 2020-01-14
Export started: SCAN_Kemole_Gulch_S1_2020-01-14_DESCENDING
Found Sentinel-1 ASCENDING image for 2020-01-14
Export started: SCAN_Kemole_Gulch_S1_2020-01-14_ASCENDING


In [ ]:
csv_path = "daily_ave_smap/VDS_Thanatpin_smap.csv"
df = pd.read_csv(csv_path)

valid_dates = df.dropna(subset='smap_soil_moisture_am')

for index, row in valid_dates.iterrows():
    print(row['time'])

2020-02-27
2020-03-03
2020-03-06
2020-03-11
2020-03-14
2020-03-19
2020-03-22
2020-03-27
2020-03-30
2020-04-04
2020-04-07
2020-04-12
2020-04-20
2020-04-23
2020-04-28
2020-05-01
2020-05-06
2020-05-09
2020-05-14
2020-05-17
2020-05-22
2020-05-25
2020-05-30
2020-06-02
2020-06-07
2020-06-10
2020-06-15
2020-06-18
2020-06-23
2020-06-26
2020-07-01
2020-07-04
2020-07-09
2020-07-12
2020-07-17
2020-07-20
2020-07-25
2020-07-28
2020-08-02
2020-08-05
2020-08-10
2020-08-13
2020-08-18
2020-08-21
2020-08-23
2020-08-26
2020-08-29
2020-09-03
2020-09-06
2020-10-29
2020-11-01
2020-11-06
2020-11-09
2020-11-14
2020-11-17
2020-11-22
2020-11-25
2020-11-30
2020-12-08
2020-12-11
2020-12-16
2020-12-19
2020-12-24
2020-12-27
